In [1]:
import torch

In [2]:
import sys

sys.path.insert(0, '/Users/xgrmir/Documents/Deeplay/deeplay')

import deeplay as dl
print(dl.__file__)

c:\Users/xgrmir/Documents/Deeplay/deeplay\deeplay\__init__.py


In [3]:
from deeplay import (
    GraphConvolutionalNeuralNetwork,
    GraphToGlobalMPM,
    GraphToNodeMPM,
    GraphToEdgeMPM,
    GraphToEdgeMAGIK,
    MessagePassingNeuralNetwork,
    ResidualMessagePassingNeuralNetwork,
    MultiLayerPerceptron,
    dense_laplacian_normalization,
    Sum,
    WeightedSum,
    Mean,
    Prod,
    Min,
    Max,
    Layer,
    GlobalMeanPool,
    GlobalGraphPooling,
    GlobalGraphUpsampling,
    MinCutPooling,
    MinCutUpsampling,
    GraphEncoder,
    GraphDecoder,
    GraphEncoderBlock,
    GraphDecoderBlock,
)

## Global pool

In [4]:
global_pool = GlobalGraphPooling()
global_pool = global_pool.build()

In [5]:
inp = {}
inp["x"] = torch.randn(3, 2)
inp["batch"] = torch.zeros(3, dtype=int)
inp["edge_index"] = torch.tensor([[0, 1, 1, 2, 1], [1, 0, 2, 1, 0]])
out = global_pool(inp)

In [6]:
out

{'x': tensor([[ 1.6337, -1.7212]]),
 'batch': tensor([0]),
 'edge_index': tensor([[0, 1, 1, 2, 1],
         [1, 0, 2, 1, 0]]),
 's': tensor([[1.],
         [1.],
         [1.]])}

In [7]:
out['x'].shape

torch.Size([1, 2])

In [8]:
out['s'].shape

torch.Size([3, 1])

In [9]:
global_upsampling = GlobalGraphUpsampling()
global_upsampling = global_upsampling.build()

In [10]:
inp = {}
inp["x"] = torch.randn(1, 2)
inp["s"] = torch.ones((3,1))
out = global_upsampling(inp)

In [11]:
out['x'].shape

torch.Size([3, 2])

## Mincut

In [46]:
mincut = MinCutPooling(num_clusters = 2, hidden_features = [5])
mincut = mincut.build()

In [47]:
mincut

MinCutPooling(
  (select): MultiLayerPerceptron(
    (blocks): LayerList(
      (0): LinearBlock(
        (layer): LazyLinear(in_features=0, out_features=5, bias=True)
        (activation): ReLU()
      )
      (1): LinearBlock(
        (layer): Linear(in_features=5, out_features=2, bias=True)
        (activation): Softmax(dim=1)
      )
    )
  )
  (batch_compatible): ClusterMatrixForBatch()
  (mincut_loss): MinCutLoss()
  (reduce): Reduce()
  (connect): Connect()
  (red_self_con): ReduceSelfConnection()
  (sparse): SparseEdgeIndex()
)

In [48]:
inp = {}
inp["x"] = torch.randn(3, 1)
inp["batch"] = torch.zeros(3, dtype=int)
inp["edge_index"] = torch.tensor([[0, 1, 1, 2, 1], [1, 0, 2, 1, 0]])
out = mincut(inp)

In [49]:
out

{'x': tensor([[-1.4034],
         [-1.7131]], grad_fn=<MmBackward0>),
 'batch': tensor([0, 0]),
 'edge_index': tensor(indices=tensor([[0, 1],
                        [1, 0]]),
        values=tensor([0.1269, 0.1285]),
        size=(2, 2), nnz=2, layout=torch.sparse_coo, grad_fn=<ToSparseBackward1>),
 's': tensor([[0.4595, 0.5405],
         [0.4439, 0.5561],
         [0.4480, 0.5520]], grad_fn=<ViewBackward0>),
 'L_cut': tensor(-1.0003, grad_fn=<DivBackward0>),
 'L_ortho': tensor(0.7652, grad_fn=<LinalgVectorNormBackward0>)}

In [50]:
out['s'].shape

torch.Size([3, 2])

In [54]:
torch.sum(out['s'], axis=1)

tensor([1., 1., 1.], grad_fn=<SumBackward1>)

In [17]:
out['edge_index']

tensor(indices=tensor([[0, 1],
                       [1, 0]]),
       values=tensor([0.1209, 0.1388]),
       size=(2, 2), nnz=2, layout=torch.sparse_coo, grad_fn=<ToSparseBackward1>)

Add test on different configurations with the layers in MinCut?

In [18]:
mincut_upsample = MinCutUpsampling()
mincut_upsample = mincut_upsample.build()

In [19]:
inp = {}
inp["x"] = torch.randn(2, 1)
inp["batch"] = torch.zeros(2, dtype=int)
inp['s'] = torch.tensor([
    [1.0, 0],
    [0, 1.0],
    [1.0, 0]
])
inp["edge_index_pool"] = torch.tensor([[0, 1], [1, 0]])
out = mincut_upsample(inp)

In [20]:
out

{'x': tensor([[1.3595],
         [0.2606],
         [1.3595]]),
 'batch': tensor([0, 0]),
 's': tensor([[1., 0.],
         [0., 1.],
         [1., 0.]]),
 'edge_index_pool': tensor([[0, 1],
         [1, 0]]),
 'edge_index': tensor([[0., 1., 0.],
         [1., 0., 1.],
         [0., 1., 0.]])}

In [21]:
out['edge_index']

tensor([[0., 1., 0.],
        [1., 0., 1.],
        [0., 1., 0.]])

In [22]:
a_pool = torch.tensor([[0, 1], [1, 0]])

In [23]:
s = torch.tensor([
    [1, 0],
    [0, 1],
    [0.8, 0.2]
])

In [24]:
a_pool = torch.sparse_coo_tensor(
                        a_pool,
                        torch.ones(a_pool.size(1)),
                        ((s.T).size(0),) * 2,
                        device=a_pool.device,
                    )
a = torch.spmm(a_pool, s.T)

In [25]:
a_pool

tensor(indices=tensor([[0, 1],
                       [1, 0]]),
       values=tensor([1., 1.]),
       size=(2, 2), nnz=2, layout=torch.sparse_coo)

In [26]:
a

tensor([[0.0000, 1.0000, 0.2000],
        [1.0000, 0.0000, 0.8000]])

In [27]:
s.T

tensor([[1.0000, 0.0000, 0.8000],
        [0.0000, 1.0000, 0.2000]])

In [28]:
mincut = MinCutPooling(num_clusters=2, hidden_features=[5], reduce_self_connection=False)
mincut = mincut.build()

In [29]:
inp = {}
inp["x"] = torch.randn(3, 1)
inp["batch"] = torch.zeros(3, dtype=int)
inp["edge_index"] = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]])
out = mincut(inp)

In [30]:
out

{'x': tensor([[0.9204],
         [1.3934]], grad_fn=<MmBackward0>),
 'batch': tensor([0, 0]),
 'edge_index': tensor(indices=tensor([[0, 0, 1, 1],
                        [0, 1, 0, 1]]),
        values=tensor([0.6270, 0.9567, 0.9567, 1.4596]),
        size=(2, 2), nnz=4, layout=torch.sparse_coo, grad_fn=<ToSparseBackward1>),
 's': tensor([[0.3918, 0.6082],
         [0.3950, 0.6050],
         [0.4019, 0.5981]], grad_fn=<ViewBackward0>),
 'L_cut': tensor(-0.9999, grad_fn=<DivBackward0>),
 'L_ortho': tensor(0.7653, grad_fn=<LinalgVectorNormBackward0>)}

## Graph Encoder and decoder

#### blocks

In [31]:
encoder_block = GraphEncoderBlock(in_features=1, out_features=4, num_clusters=2)
encoder_block = encoder_block.build()

In [32]:
encoder_block

GraphEncoderBlock(
  (gcn): GraphConvolutionalNeuralNetwork(
    (normalize): sparse_laplacian_normalization()
    (blocks): LayerList(
      (0): TransformPropagateUpdate(
        (transform): Linear(in_features=1, out_features=4, bias=True)
        (propagate): Propagate()
        (update): ReLU()
      )
    )
  )
  (pool): MinCutPooling(
    (select): MultiLayerPerceptron(
      (blocks): LayerList(
        (0): LinearBlock(
          (layer): LazyLinear(in_features=0, out_features=4, bias=True)
          (activation): ReLU()
        )
        (1): LinearBlock(
          (layer): Linear(in_features=4, out_features=2, bias=True)
          (activation): Softmax(dim=1)
        )
      )
    )
    (batch_compatible): ClusterMatrixForBatch()
    (mincut_loss): MinCutLoss()
    (reduce): Reduce()
    (connect): Connect()
    (red_self_con): ReduceSelfConnection()
    (sparse): SparseEdgeIndex()
  )
)

In [33]:
inp = {}
inp["x"] = torch.randn(3, 1)
inp["batch"] = torch.zeros(3, dtype=int)
inp["edge_index"] = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]])
out = encoder_block(inp)

In [34]:
out

{'x': tensor([[0.4139, 0.0000, 1.0564, 1.0303],
         [0.1099, 0.0000, 0.2809, 0.2738]], grad_fn=<MmBackward0>),
 'batch': tensor([0, 0]),
 'edge_index': tensor([[0, 1, 1, 2],
         [1, 0, 2, 1]]),
 'laplacian': tensor(indices=tensor([[0, 0, 1, 1, 1, 2, 2],
                        [0, 1, 0, 1, 2, 1, 2]]),
        values=tensor([0.5000, 0.4082, 0.4082, 0.3333, 0.4082, 0.4082, 0.5000]),
        size=(3, 3), nnz=7, layout=torch.sparse_coo),
 's': tensor([[0.7871, 0.2129],
         [0.7905, 0.2095],
         [0.7918, 0.2082]], grad_fn=<ViewBackward0>),
 'L_cut': tensor(-1.0000, grad_fn=<DivBackward0>),
 'L_ortho': tensor(0.7654, grad_fn=<LinalgVectorNormBackward0>),
 'edge_index_pool': tensor(indices=tensor([[0, 1],
                        [1, 0]]),
        values=tensor([0.0838, 0.0838]),
        size=(2, 2), nnz=2, layout=torch.sparse_coo, grad_fn=<ToSparseBackward1>)}

In [35]:
out['s'].shape

torch.Size([3, 2])

In [36]:
decoder_block = GraphDecoderBlock(in_features=1, out_features=4)
decoder_block = decoder_block.build()

inp = {}
inp["x"] = torch.randn(2, 1)
inp["batch"] = torch.zeros(2, dtype=int)
inp["edge_index_pool"] = torch.tensor([[0, 1], [1, 0]])
inp["edge_index"] = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]])
inp['s'] = torch.tensor([
    [1.0, 0],
    [0, 1.0],
    [1.0, 0]
])
out = decoder_block(inp)

In [37]:
out

{'x': tensor([[0.0000, 0.0000, 0.4061, 0.0000],
         [0.0000, 0.0000, 0.5079, 0.0000],
         [0.0000, 0.0000, 0.4061, 0.0000]], grad_fn=<ReluBackward0>),
 'batch': tensor([0, 0]),
 'edge_index_pool': tensor([[0, 1],
         [1, 0]]),
 'edge_index': tensor([[0, 1, 1, 2],
         [1, 0, 2, 1]]),
 's': tensor([[1., 0.],
         [0., 1.],
         [1., 0.]]),
 '-': tensor([[0., 1., 0.],
         [1., 0., 1.],
         [0., 1., 0.]]),
 'laplacian': tensor(indices=tensor([[0, 0, 1, 1, 1, 2, 2],
                        [0, 1, 0, 1, 2, 1, 2]]),
        values=tensor([0.5000, 0.4082, 0.4082, 0.3333, 0.4082, 0.4082, 0.5000]),
        size=(3, 3), nnz=7, layout=torch.sparse_coo)}

In [38]:
out['x'].shape

torch.Size([3, 4])

#### Encoder and decoder

In [39]:
graph_encoder = GraphEncoder(hidden_features=2, num_blocks=2, num_clusters=[2,1])
graph_encoder = graph_encoder.build()

In [40]:
inp = {}
inp["x"] = torch.randn(4, 2)
inp["batch"] = torch.zeros(4, dtype=int)
inp["edge_index"] = torch.tensor([[0, 1, 1, 2, 1, 3], [1, 0, 2, 1, 3, 1]])
inp["edge_attr"] = torch.randn(6, 1)
out = graph_encoder(inp)

In [41]:
len(graph_encoder.blocks)

2

In [42]:
out

{'x': tensor([[0.8827, 0.0000]], grad_fn=<MmBackward0>),
 'batch': tensor([0, 0, 0, 0]),
 'edge_index': tensor([[0, 1, 1, 2, 1, 3],
         [1, 0, 2, 1, 3, 1]]),
 'edge_attr': tensor([[0.0046, 0.0058],
         [0.5006, 0.0829],
         [0.3284, 0.0937],
         [0.1720, 0.0512],
         [0.3950, 0.0706],
         [0.5021, 0.0839]], grad_fn=<ReluBackward0>),
 'aggregate': tensor([[0.5006, 0.0829],
         [0.6788, 0.1409],
         [0.3284, 0.0937],
         [0.3950, 0.0706]], grad_fn=<ScatterAddBackward0>),
 'laplacian': tensor(indices=tensor([[0, 0, 1, 1],
                        [0, 1, 0, 1]]),
        values=tensor([0.5000, 0.5000, 0.5000, 0.5000]),
        size=(2, 2), nnz=4, layout=torch.sparse_coo),
 's_0': tensor([[0.4963, 0.5037],
         [0.4963, 0.5037],
         [0.4963, 0.5037],
         [0.4963, 0.5037]], grad_fn=<ViewBackward0>),
 'batch_1': tensor([0, 0]),
 'L_cut_0': tensor(-1., grad_fn=<DivBackward0>),
 'L_ortho_0': tensor(0.7654, grad_fn=<LinalgVectorNormBackwa

In [43]:
out['edge_index_1'].shape

torch.Size([2, 2])

In [44]:
graph_decoder = GraphDecoder(hidden_features=2, num_blocks=2, output_node_dim=2, output_edge_dim=1)
graph_decoder = graph_decoder.build()

print(len(graph_decoder.blocks))

inp = {}
inp["x"] = torch.randn(1, 2)
inp["batch"] = torch.zeros(1, dtype=int)
inp["edge_index_1"] = torch.tensor([[0, 1], [1, 0]])
inp["edge_index"] = torch.tensor([[0, 1, 1, 2, 1, 3], [1, 0, 2, 1, 3, 1]])
inp['s_1'] = torch.ones((2,1))
inp['s_0'] = torch.tensor([[1.0, 0], [0, 1.0], [0, 1.0], [1.0, 0]])
out = graph_decoder(inp)

2


In [45]:
out['edge_attr']

tensor([[0.5838],
        [0.5838],
        [0.5838],
        [0.5838],
        [0.5838],
        [0.5838]], grad_fn=<AddmmBackward0>)